In [1]:
using Pkg
# Activate environment that has ProgressiveHedging installed
Pkg.activate(joinpath(@__DIR__, ".."))
Pkg.status()

 Activating environment at `~/jdev/ProgressiveHedging/Project.toml`


Project ProgressiveHedging v0.4.0
Status `~/jdev/ProgressiveHedging/Project.toml`
  [a93c6f00] DataFrames v0.22.5
  [b6b21f68] Ipopt v0.6.5
  [4076af6c] JuMP v0.21.6
  [b8f27783] MathOptInterface v0.9.20
  [a759f4b9] TimerOutputs v0.5.8
  [8ba89e20] Distributed
  [de0858da] Printf


In [2]:
using Distributed
const WORKERS = 2 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(joinpath(@__DIR__, "..")))
    end
end

@everywhere using ProgressiveHedging
@everywhere const PH = ProgressiveHedging

@everywhere using GLPK
@everywhere using Ipopt
@everywhere using JuMP
@everywhere using Xpress

@everywhere using Random

Adding 2 worker processes.
      From worker 2:	 Activating environment at `~/jdev/ProgressiveHedging/Project.toml`
      From worker 3:	 Activating environment at `~/jdev/ProgressiveHedging/Project.toml`


┌ Info: Xpress: Found license file /Users/jmaack/MyApps/xpress/xpressmp/bin/xpauth.xpr
└ @ Xpress /Users/jmaack/.julia/packages/Xpress/ab5Cg/src/license.jl:42
┌ Info: Xpress: Development license detected.
└ @ Xpress /Users/jmaack/.julia/packages/Xpress/ab5Cg/src/license.jl:84
[ Info: Xpress: Found license file /Users/jmaack/MyApps/xpress/xpressmp/bin/xpauth.xpr
[ Info: Xpress: Development license detected.
[ Info: Xpress: Found license file /Users/jmaack/MyApps/xpress/xpressmp/bin/xpauth.xpr
[ Info: Xpress: Development license detected.


### Model Construction Function Definition

In [3]:
function two_stage_int(scenario_id::PH.ScenarioID)
    model = JuMP.Model(()->Xpress.Optimizer())
    JuMP.set_optimizer_attribute(model, "OUTPUTLOG", 0)
#     JuMP.set_optimizer_attribute(model, "tol", 1e-12)
#     JuMP.set_optimizer_attribute(model, "acceptable_tol", 1e-12)

    scen = PH.value(scenario_id)

    ref = JuMP.@variable(model, x)
    stage1 = [ref]
    push!(stage1, JuMP.@variable(model, k, binary=true))
    JuMP.@constraint(model, x <= 1000.0*k)
    JuMP.@constraint(model, x >= 1.0*k)

    ref = JuMP.@variable(model, y >= 0.0)
    stage2 = [ref]

    # val = scen == 0 ? 10.0 : 3.0
    Random.seed!(scen)
    val = 10.0*rand()

    JuMP.@constraint(model, x + y == val)

    # c_y = scen == 0 ? 1.5 : 2.0
    c_y = (scen % 2 == 0 ? 2.0 : 5.0)
    JuMP.@objective(model, Min, 2.0*x + c_y * y + 1.0*k)

    return PH.JuMPSubproblem(model, scenario_id, Dict(PH.stid(1) => stage1,
                                                      PH.stid(2) => stage2)
                             )
end


two_stage_int (generic function with 1 method)

In [4]:
N = 8

8

### Extensive Form Solve

In [5]:
ef_model = @time PH.solve_extensive(PH.two_stage_tree(N),
    two_stage_int, 
    ()->Xpress.Optimizer(),
    opt_args=(OUTPUTLOG=0,)
)
println(ef_model)

 21.058309 seconds (59.60 M allocations: 2.998 GiB, 6.92% gc time)
Min 2 x_{0,1,2,3,4,5,6,7} + 0.625 y_{3} + k_{0,1,2,3,4,5,6,7} + 0.25 y_{6} + 0.625 y_{1} + 0.25 y_{4} + 0.25 y_{2} + 0.625 y_{7} + 0.25 y_{0} + 0.625 y_{5}
Subject to
 y_{3} + x_{0,1,2,3,4,5,6,7} = 8.116984049958615
 x_{0,1,2,3,4,5,6,7} + y_{6} = 5.114774368570256
 x_{0,1,2,3,4,5,6,7} + y_{1} = 2.3603334566204692
 x_{0,1,2,3,4,5,6,7} + y_{4} = 6.800792359357411
 x_{0,1,2,3,4,5,6,7} + y_{2} = 3.6679641243992434
 x_{0,1,2,3,4,5,6,7} + y_{7} = 8.124385822007527
 x_{0,1,2,3,4,5,6,7} + y_{0} = 8.236475079774124
 x_{0,1,2,3,4,5,6,7} + y_{5} = 5.303646605536807
 x_{0,1,2,3,4,5,6,7} - k_{0,1,2,3,4,5,6,7} ≥ 0.0
 x_{0,1,2,3,4,5,6,7} - 1000 k_{0,1,2,3,4,5,6,7} ≤ 0.0
 y_{3} ≥ 0.0
 y_{6} ≥ 0.0
 y_{1} ≥ 0.0
 y_{4} ≥ 0.0
 y_{2} ≥ 0.0
 y_{7} ≥ 0.0
 y_{0} ≥ 0.0
 y_{5} ≥ 0.0
 k_{0,1,2,3,4,5,6,7} binary



In [6]:
println(JuMP.termination_status(ef_model))
println(JuMP.objective_value(ef_model))
for var in JuMP.all_variables(ef_model)
    println("$var = $(JuMP.value(var))")
end

OPTIMAL
18.355345006921688
y_{3} = 5.756650593338146
x_{0,1,2,3,4,5,6,7} = 2.3603334566204692
k_{0,1,2,3,4,5,6,7} = 1.0
y_{6} = 2.7544409119497866
y_{1} = 0.0
y_{4} = 4.440458902736942
y_{2} = 1.3076306677787741
y_{7} = 5.764052365387058
y_{0} = 5.8761416231536545
y_{5} = 2.9433131489163378


### PH Solve

In [7]:
using DataFrames
using Plots

In [8]:
st = PH.two_stage_tree(N)
(n, err, rerr, obj, soln, phd_sc) = PH.solve(st,
                                     two_stage_int,
                                     PH.ScalarPenaltyParameter(1.0),
                                     atol=1e-8, rtol=1e-12, max_iter=500, report=0,
                                     save_iterates=1, save_residuals=1)
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

LoadError: [91mTaskFailedException:[39m
[91m[91mOn worker 2:[39m[39m
[91m[91m[91mUndefVarError: #two_stage_int not defined[39m[39m[39m
[91m[91mdeserialize_datatype at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1252[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:826[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:833[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1350[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:837[39m[39m
[91m[91mdeseria

In [9]:
aobj = PH.retrieve_aug_obj_value(phd_sc)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [10]:
soln

LoadError: [91mUndefVarError: soln not defined[39m

In [11]:
xhats = PH.retrieve_xhat_history(phd_sc)
p = plot()
for n in names(select(xhats, Not(:iteration)))
    plot!(p, xhats[:,:iteration], xhats[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [12]:
xs = PH.retrieve_no_hat_history(phd_sc)
p = plot()
for n in names(select(xs, DataFrames.Not(:iteration)))
    plot!(p, xs[:,:iteration], xs[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [13]:
ws = PH.retrieve_w_history(phd_sc)
p = plot()
for n in names(select(ws, DataFrames.Not(:iteration)))
    plot!(p, ws[:,:iteration], ws[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [14]:
res_scalar = PH.residuals(phd_sc)
rel_scalar = PH.relative_residuals(phd_sc)
plot(res_scalar, yscale=:log10)
plot!(rel_scalar, yscale=:log10)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [15]:
(xhat_sq, x_sq) = PH.residual_components(phd_sc)
plot(sqrt.(xhat_sq), yscale=:log10)
plot!(sqrt.(x_sq), yscale=:log10)

LoadError: [91mUndefVarError: phd_sc not defined[39m

In [16]:
st = PH.two_stage_tree(N)
(n, err, rerr, obj, soln, phd_pr) = PH.solve(st,
                                    two_stage_int,
                                    PH.ProportionalPenaltyParameter(1.0),
                                    atol=1e-8, rtol=1e-12, max_iter=500, report=0,
                                    save_iterates=1, save_residuals=1)
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

LoadError: [91mTaskFailedException:[39m
[91m[91mOn worker 2:[39m[39m
[91m[91m[91mUndefVarError: #two_stage_int not defined[39m[39m[39m
[91m[91mdeserialize_datatype at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1252[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:826[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:833[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1350[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:837[39m[39m
[91m[91mdeseria

In [17]:
aobj = PH.retrieve_aug_obj_value(phd_pr)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [18]:
phd_pr.r.penalties

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [19]:
xhats = PH.retrieve_xhat_history(phd_pr)
p = plot()
for n in names(select(xhats, Not(:iteration)))
    plot!(p, xhats[:,:iteration], xhats[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [20]:
xs = PH.retrieve_no_hat_history(phd_pr)
p = plot()
for n in names(select(xs, DataFrames.Not(:iteration)))
    plot!(p, xs[:,:iteration], xs[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [21]:
ws = PH.retrieve_w_history(phd_pr)
p = plot()
for n in names(select(ws, DataFrames.Not(:iteration)))
    plot!(p, ws[:,:iteration], ws[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [22]:
res_prop = PH.residuals(phd_pr)
rel_prop = PH.relative_residuals(phd_pr)
plot(res_prop, yscale=:log10)
plot!(rel_prop, yscale=:log10)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [23]:
(xhat_sq, x_sq) = PH.residual_components(phd_pr)
plot(sqrt.(xhat_sq), yscale=:log10)
plot!(sqrt.(x_sq), yscale=:log10)

LoadError: [91mUndefVarError: phd_pr not defined[39m

In [24]:
st = PH.two_stage_tree(N)
(n, err, rerr, obj, soln, phd_sp) = PH.solve(st,
                                    two_stage_int,
                                    PH.SEPPenaltyParameter(),
                                    atol=1e-8, rtol=1e-12, max_iter=500, report=0,
                                    save_iterates=1, save_residuals=1)
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

LoadError: [91mTaskFailedException:[39m
[91m[91mOn worker 2:[39m[39m
[91m[91m[91mUndefVarError: #two_stage_int not defined[39m[39m[39m
[91m[91mdeserialize_datatype at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1252[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:826[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:773[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:833[39m[39m
[91m[91mdeserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:1350[39m[39m
[91m[91mhandle_deserialize at /Users/jmaack/MyApps/julia_v1.5/usr/share/julia/stdlib/v1.5/Serialization/src/Serialization.jl:837[39m[39m
[91m[91mdeseria

In [25]:
aobj = PH.retrieve_aug_obj_value(phd_sp)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

LoadError: [91mUndefVarError: phd_sp not defined[39m

In [26]:
xhats = PH.retrieve_xhat_history(phd_sp)
p = plot()
for n in names(select(xhats, Not(:iteration)))
    plot!(p, xhats[:,:iteration], xhats[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sp not defined[39m

In [27]:
xs = PH.retrieve_no_hat_history(phd_sp)
p = plot()
for n in names(select(xs, DataFrames.Not(:iteration)))
    plot!(p, xs[:,:iteration], xs[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sp not defined[39m

In [28]:
ws = PH.retrieve_w_history(phd_sp)
p = plot()
for n in names(select(ws, DataFrames.Not(:iteration)))
    plot!(p, ws[:,:iteration], ws[:,n], label=n)
end
display(p)

LoadError: [91mUndefVarError: phd_sp not defined[39m

In [29]:
res_prop = PH.residuals(phd_sp)
rel_prop = PH.relative_residuals(phd_sp)
plot(res_prop, yscale=:log10)
plot!(rel_prop, yscale=:log10)

LoadError: [91mUndefVarError: phd_sp not defined[39m

In [30]:
(xhat_sq, x_sq) = PH.residual_components(phd_sp)
plot(sqrt.(xhat_sq), yscale=:log10)
plot!(sqrt.(x_sq), yscale=:log10)

LoadError: [91mUndefVarError: phd_sp not defined[39m